In [76]:
import pandas as pd
from numpy import nan, infty

# Gorilla assessment

This is the assessment for an job application for Gorilla. 



## Importing the data

In [2]:
consumption=pd.read_excel("data.xlsx", sheet_name="Forecasted Consumption")
meters=pd.read_excel("data.xlsx", sheet_name="Meter List")
rates=pd.read_excel("data.xlsx", sheet_name="Rates")

In [3]:
for x in (consumption,rates,meters): 
    print(x.dtypes)

Date        datetime64[ns]
Meter ID             int64
kWh                float64
dtype: object
Date                     datetime64[ns]
Exit Zone                        object
Annual Quantity (Min)             int64
Annual Quantity (Max)           float64
Rate (p/kWh)                    float64
dtype: object
Meter ID                  int64
Exit Zone                object
Annual Quantity (kWh)     int64
dtype: object


In [4]:
rates

,Date,Exit Zone,Annual Quantity (Min),Annual Quantity (Max),Rate (p/kWh)
0,2020-04-01,EA1,0,73200.0,0.028700
1,2020-04-01,EA2,0,73200.0,0.028700
2,2020-04-01,EA3,0,73200.0,0.028700
3,2020-04-01,EA4,0,73200.0,0.028700
4,2020-04-01,EM1,0,73200.0,0.028700
...,...,...,...,...,...
1021,2024-04-01,WA1,732000,NaN,0.384755
1022,2024-04-01,WA2,732000,NaN,0.384755
1023,2024-04-01,WM1,732000,NaN,0.445482
1024,2024-04-01,WM2,732000,NaN,0.445482


In [5]:
consumption

,Date,Meter ID,kWh
0,2020-10-01,1000000603,28.782474
1,2020-10-01,10588707,126.367711
2,2020-10-01,10626610,326.240595
3,2020-10-02,1000000603,26.690797
4,2020-10-02,10588707,118.322449
...,...,...,...
2185,2022-09-29,10588707,77.744203
2186,2022-09-29,10626610,270.357196
2187,2022-09-30,1000000603,17.927143
2188,2022-09-30,10588707,73.335098


In [6]:
meters

,Meter ID,Exit Zone,Annual Quantity (kWh)
0,1000000603,SC1,12337
1,10588707,EM2,75123
2,10626610,NW1,151779


## Excersise 1. Transportation Distribution Charge

> A **Transportation Distribution Charge** is a charge levied by the Distribution companies for the use of their lower pressure pipelines. The charge covers the cost of physically transporting the gas through the pipeline.  The cost is variable in time and is determined by the Exit Zone (a regional code) and the (estimated) rolling consumption quantity of the meter.

1. Calculate the **total cost per meter** by summing the costs per day for the full period of
   the forecast (2020-10-01 to 2022-09-30) and converting to £ (1p = 0.01£),
2. Calculate the **total consumption** by summing the forecasted consumption for the full
   period.

First, we need a merge on the data, with the necessary data fields: meter id, correct rate and consumption.

### First merge: add meter info to the consumption
To connect the `consumption` to the `rates`, we need to have the field _Exit zone_, so first we merge with `meters`. In this case, this is a left join, because we only need meters that are in the `consumption` table (and we don't want to throw away consumption data, should the correct meter not be in the `meters` table). 

In [121]:
merged_data=pd.merge(consumption, meters, on="Meter ID", how="left")
merged_data.sort_values("Date")

,Date,Meter ID,kWh,Exit Zone,Annual Quantity (kWh)
0,2020-10-01,1000000603,28.782474,SC1,12337
1,2020-10-01,10588707,126.367711,EM2,75123
2,2020-10-01,10626610,326.240595,NW1,151779
3,2020-10-02,1000000603,26.690797,SC1,12337
4,2020-10-02,10588707,118.322449,EM2,75123
...,...,...,...,...,...
2184,2022-09-29,1000000603,19.263599,SC1,12337
2186,2022-09-29,10626610,270.357196,NW1,151779
2188,2022-09-30,10588707,73.335098,EM2,75123
2187,2022-09-30,1000000603,17.927143,SC1,12337


### Second merge: the rates
Now let's take a look at the `rates` table.

In [65]:
rates.head()

,Date,Exit Zone,Annual Quantity (Min),Annual Quantity (Max),Rate (p/kWh)
0,2020-04-01,EA1,0,73200.0,0.0287
1,2020-04-01,EA2,0,73200.0,0.0287
2,2020-04-01,EA3,0,73200.0,0.0287
3,2020-04-01,EA4,0,73200.0,0.0287
4,2020-04-01,EM1,0,73200.0,0.0287


In [87]:
rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026 entries, 0 to 1025
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Date                   1026 non-null   datetime64[ns]
 1   Exit Zone              1026 non-null   object        
 2   Annual Quantity (Min)  1026 non-null   int64         
 3   Annual Quantity (Max)  684 non-null    float64       
 4   Rate (p/kWh)           1026 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 40.2+ KB


Let's look at all available data for one day.

In [125]:
rates[rates.Date=="2020-04-01"].sort_values(["Exit Zone", "Annual Quantity (Min)"])

,Date,Exit Zone,Annual Quantity (Min),Annual Quantity (Max),Rate (p/kWh)
0,2020-04-01,EA1,0,73200.0,0.0287
38,2020-04-01,EA1,73200,732000.0,0.0228
76,2020-04-01,EA1,732000,inf,0.1820
1,2020-04-01,EA2,0,73200.0,0.0287
39,2020-04-01,EA2,73200,732000.0,0.0228
...,...,...,...,...,...
74,2020-04-01,WM2,73200,732000.0,0.0292
112,2020-04-01,WM2,732000,inf,0.3927
37,2020-04-01,WM3,0,73200.0,0.0326
75,2020-04-01,WM3,73200,732000.0,0.0292


In order to compare later on, I replace the NaNs for max annual rate with ininity, because that's what the NaNs mean here.

In [98]:
rates["Annual Quantity (Max)"]=rates["Annual Quantity (Max)"].fillna(infty)

Next merge, this is a many to many relationship for date and exit zone, between this table and the `rates` dataframe.

In [109]:
merged_rates=pd.merge(merged_data, rates, how="left", on = ["Date", "Exit Zone"])
merged_rates.sort_values("Date")

,Date,Meter ID,kWh,Exit Zone,Annual Quantity (kWh),Annual Quantity (Min),Annual Quantity (Max),Rate (p/kWh)
0,2020-10-01,1000000603,28.782474,SC1,12337,0.0,73200.0,0.0333
1,2020-10-01,1000000603,28.782474,SC1,12337,73200.0,732000.0,0.0297
2,2020-10-01,1000000603,28.782474,SC1,12337,732000.0,inf,0.2431
3,2020-10-01,10588707,126.367711,EM2,75123,0.0,73200.0,0.0287
4,2020-10-01,10588707,126.367711,EM2,75123,73200.0,732000.0,0.0228
...,...,...,...,...,...,...,...,...
2208,2022-09-29,1000000603,19.263599,SC1,12337,NaN,NaN,NaN
2210,2022-09-29,10626610,270.357196,NW1,151779,NaN,NaN,NaN
2212,2022-09-30,10588707,73.335098,EM2,75123,NaN,NaN,NaN
2211,2022-09-30,1000000603,17.927143,SC1,12337,NaN,NaN,NaN


So apparently, not all dates match exactly, therefore there are a lot of NaNs...

In [110]:
merged_rates.isna().sum()

Date                        0
Meter ID                    0
kWh                         0
Exit Zone                   0
Annual Quantity (kWh)       0
Annual Quantity (Min)    2178
Annual Quantity (Max)    2178
Rate (p/kWh)             2178
dtype: int64

Not every date that's in the `consumption` table is represented in the `rates` table. Rather than throwing away everything, we want to use the closest previous rate. Luckily, there is `pd.merge_asof()` for that.

In [142]:
merged_rates = pd.merge_asof(
    merged_data.sort_values(
        "Date"
    ),  # documentation says that both dataframes should be sorted for this
    rates.sort_values("Date"),
    by="Exit Zone",  # also merge on exit zone, before the asof merge is performed
    on="Date",
    direction="backward",  # "Selects the last row in the right DataFrame whose 'on' key is less than or equal to the left's key.""
)
merged_rates.sort_values("Date")

,Date,Meter ID,kWh,Exit Zone,Annual Quantity (kWh),Annual Quantity (Min),Annual Quantity (Max),Rate (p/kWh)
0,2020-10-01,1000000603,28.782474,SC1,12337,0,73200.0,0.033300
1,2020-10-01,10588707,126.367711,EM2,75123,73200,732000.0,0.022800
2,2020-10-01,10626610,326.240595,NW1,151779,0,73200.0,0.033900
3,2020-10-02,1000000603,26.690797,SC1,12337,0,73200.0,0.033300
4,2020-10-02,10588707,118.322449,EM2,75123,73200,732000.0,0.022800
...,...,...,...,...,...,...,...,...
2184,2022-09-29,10588707,77.744203,EM2,75123,73200,732000.0,0.024358
2186,2022-09-29,10626610,270.357196,NW1,151779,0,73200.0,0.036022
2188,2022-09-30,1000000603,17.927143,SC1,12337,0,73200.0,0.034706
2187,2022-09-30,10588707,73.335098,EM2,75123,73200,732000.0,0.024358


Unfortunately, now we lost the annual quantity intervals, as merge_asof only does a left join (and thus selects a interval arbitrarily). For the interval, we actually should also do a `merge_asof()`. Starting with that would only keep one date → same problem. 

The only solution I can now think of for now is to first do a full outer merge between consumption and rates, and then filter afterwards. So let's start over.

In [173]:
full_merge = pd.merge(
    merged_data, rates, on="Exit Zone", how="left", suffixes=("_consumption", "_rates")
)
full_merge

,Date_consumption,Meter ID,kWh,Exit Zone,Annual Quantity (kWh),Date_rates,Annual Quantity (Min),Annual Quantity (Max),Rate (p/kWh)
0,2020-10-01,1000000603,28.782474,SC1,12337,2020-04-01,0,73200.0,0.033300
1,2020-10-01,1000000603,28.782474,SC1,12337,2020-04-01,73200,732000.0,0.029700
2,2020-10-01,1000000603,28.782474,SC1,12337,2020-04-01,732000,inf,0.243100
3,2020-10-01,1000000603,28.782474,SC1,12337,2020-10-01,0,73200.0,0.033300
4,2020-10-01,1000000603,28.782474,SC1,12337,2020-10-01,73200,732000.0,0.029700
...,...,...,...,...,...,...,...,...,...
59125,2022-09-30,10626610,263.375674,NW1,151779,2023-10-01,73200,732000.0,0.031236
59126,2022-09-30,10626610,263.375674,NW1,151779,2023-10-01,732000,inf,0.271146
59127,2022-09-30,10626610,263.375674,NW1,151779,2024-04-01,0,73200.0,0.038320
59128,2022-09-30,10626610,263.375674,NW1,151779,2024-04-01,73200,732000.0,0.032216


We only want the rows were the annual quantity matches the correct annual rate interval.

In [174]:
full_merge_quant = full_merge.query(
    "`Annual Quantity (Max)`>=`Annual Quantity (kWh)`>=`Annual Quantity (Min)`"
)

Now we filter on the date. `merge_asof` is not possible anymore, so I use normal filtering. We need the last date from the rates. First selecting everything before the consumption date, sorting and taking the last one accomplishes this.

In [178]:
full_merge_date=full_merge_quant[
    full_merge_quant.Date_consumption >= full_merge_quant.Date_rates
].sort_values("Date_rates").groupby(["Date_consumption", "Meter ID"]).last()
full_merge_date

kWh Exit Zone  Annual Quantity (kWh)  \
Date_consumption Meter ID                                                  
2020-10-01       10588707    126.367711       EM2                  75123   
                 10626610    326.240595       NW1                 151779   
                 1000000603   28.782474       SC1                  12337   
2020-10-02       10588707    118.322449       EM2                  75123   
                 10626610    316.860290       NW1                 151779   
...                                 ...       ...                    ...   
2022-09-29       10626610    270.357196       NW1                 151779   
                 1000000603   19.263599       SC1                  12337   
2022-09-30       10588707     73.335098       EM2                  75123   
                 10626610    263.375674       NW1                 151779   
                 1000000603   17.927143       SC1                  12337   

                            Date_rates  Annual Quantity (Min)  \
Date_consumption Meter ID                                       
2020-10-01       10588707   2020-10-01                  73200   
                 10626610   2020-10-01                  73200   
                 1000000603 2020-10-01                      0   
2020-10-02       10588707   2020-10-01                  73200   
                 10626610   2020-10-01                  73200   
...                                ...                    ...   
2022-09-29       10626610   2022-04-01                  73200   
                 1000000603 2022-04-01                      0   
2022-09-30       10588707   2022-04-01                  73200   
                 10626610   2022-04-01                  73200   
                 1000000603 2022-04-01                      0   

                             Annual Quantity (Max)  Rate (p/kWh)  
Date_consumption Meter ID                                         
2020-10-01       10588707                 732000.0      0.022800  
                 10626610                 732000.0      0.028500  
                 1000000603                73200.0      0.033300  
2020-10-02       10588707                 732000.0      0.022800  
                 10626610                 732000.0      0.028500  
...                                            ...           ...  
2022-09-29       10626610                 732000.0      0.030284  
                 1000000603                73200.0      0.034706  
2022-09-30       10588707                 732000.0      0.024358  
                 10626610                 732000.0      0.030284  
                 1000000603                73200.0      0.034706  

[2190 rows x 7 columns]

Sanity check, when are the rates changed?

In [164]:
pd.unique(rates.Date)

array(['2020-04-01T00:00:00.000000000', '2020-10-01T00:00:00.000000000',
       '2021-04-01T00:00:00.000000000', '2021-10-01T00:00:00.000000000',
       '2022-04-01T00:00:00.000000000', '2022-10-01T00:00:00.000000000',
       '2023-04-01T00:00:00.000000000', '2023-10-01T00:00:00.000000000',
       '2024-04-01T00:00:00.000000000'], dtype='datetime64[ns]')

The rates are changed every April and October, so the results above seem to be correct.

At last, we can generate the requested calculations.

In [191]:
full_merge_date["Total Cost"]=full_merge_date["kWh"]*full_merge_date["Rate (p/kWh)"]
full_merge_date

kWh Exit Zone  Annual Quantity (kWh)  \
Date_consumption Meter ID                                                  
2020-10-01       10588707    126.367711       EM2                  75123   
                 10626610    326.240595       NW1                 151779   
                 1000000603   28.782474       SC1                  12337   
2020-10-02       10588707    118.322449       EM2                  75123   
                 10626610    316.860290       NW1                 151779   
...                                 ...       ...                    ...   
2022-09-29       10626610    270.357196       NW1                 151779   
                 1000000603   19.263599       SC1                  12337   
2022-09-30       10588707     73.335098       EM2                  75123   
                 10626610    263.375674       NW1                 151779   
                 1000000603   17.927143       SC1                  12337   

                            Date_rates  Annual Quantity (Min)  \
Date_consumption Meter ID                                       
2020-10-01       10588707   2020-10-01                  73200   
                 10626610   2020-10-01                  73200   
                 1000000603 2020-10-01                      0   
2020-10-02       10588707   2020-10-01                  73200   
                 10626610   2020-10-01                  73200   
...                                ...                    ...   
2022-09-29       10626610   2022-04-01                  73200   
                 1000000603 2022-04-01                      0   
2022-09-30       10588707   2022-04-01                  73200   
                 10626610   2022-04-01                  73200   
                 1000000603 2022-04-01                      0   

                             Annual Quantity (Max)  Rate (p/kWh)  Total Cost  
Date_consumption Meter ID                                                     
2020-10-01       10588707                 732000.0      0.022800    2.881184  
                 10626610                 732000.0      0.028500    9.297857  
                 1000000603                73200.0      0.033300    0.958456  
2020-10-02       10588707                 732000.0      0.022800    2.697752  
                 10626610                 732000.0      0.028500    9.030518  
...                                            ...           ...         ...  
2022-09-29       10626610                 732000.0      0.030284    8.187461  
                 1000000603                73200.0      0.034706    0.668562  
2022-09-30       10588707                 732000.0      0.024358    1.786285  
                 10626610                 732000.0      0.030284    7.976033  
                 1000000603                73200.0      0.034706    0.622179  

[2190 rows x 8 columns]

In [196]:
result=full_merge_date.groupby("Meter ID")[["kWh", "Total Cost"]].sum()
result.columns=("Total Estimated Consumption (kWh)", "Total Cost (£)")

Finally, the result can get rounded to two decimals.

In [203]:
result["Total Cost (£)"]=result["Total Cost (£)"].round(2)
result["Total Estimated Consumption (kWh)"]=result["Total Estimated Consumption (kWh)"].round(0)
result

,Total Estimated Consumption (kWh),Total Cost (£)
Meter ID,,
10588707,122450.0,2849.81
10626610,303558.0,8823.66
1000000603,24674.0,826.78


## Excersise 2: mock meters

*Write a function that generates a list of random meters of any size. Examples of valid
Exit Zones can be found in the rate table. You may randomly generate the Annual
Quantity.*

In [209]:
exit_zones=pd.unique(rates["Exit Zone"])
exit_zones

array(['EA1', 'EA2', 'EA3', 'EA4', 'EM1', 'EM2', 'EM3', 'EM4', 'LC', 'LO',
       'LS', 'LT', 'LW', 'NE1', 'NE2', 'NE3', 'NO1', 'NO2', 'NT1', 'NT2',
       'NT3', 'NW1', 'NW2', 'SC1', 'SC2', 'SC4', 'SE1', 'SE2', 'SO1',
       'SO2', 'SW1', 'SW2', 'SW3', 'WA1', 'WA2', 'WM1', 'WM2', 'WM3'],
      dtype=object)

In [207]:
meters

,Meter ID,Exit Zone,Annual Quantity (kWh)
0,1000000603,SC1,12337
1,10588707,EM2,75123
2,10626610,NW1,151779


Let's import some numpy functions.

In [234]:
from numpy.random import random, choice, randint
from typing import Tuple

In [268]:
def generate_meters(
    n: int, annual_kwh_range: Tuple[int, int] = (5000, 1_000_000)
) -> pd.DataFrame:
    """Generates a Dataframe of random meters, with length n. 
    Optionally choose the range for the random annual quantity."""
    
    zones = choice(exit_zones, n)
    aqs = randint(annual_kwh_range[0], annual_kwh_range[1], n)
    result=pd.DataFrame({"Exit Zone": zones, "Annual Quantity (kWh)": aqs})
    result.index.name="Meter ID"
    return result

In [269]:
generate_meters(10)

,Exit Zone,Annual Quantity (kWh)
Meter ID,,
0,SW3,459628
1,LS,643036
2,SO2,99781
3,EM1,816034
4,SC2,875592
5,EA3,994194
6,SW3,963310
7,WM2,98500
8,SW2,283511


## Excersise 3: Mock consumption data
*Write a function that generates mock consumption data given a list of meters and a
start and end date.*

In [239]:
consumption

,Date,Meter ID,kWh
0,2020-10-01,1000000603,28.782474
1,2020-10-01,10588707,126.367711
2,2020-10-01,10626610,326.240595
3,2020-10-02,1000000603,26.690797
4,2020-10-02,10588707,118.322449
...,...,...,...
2185,2022-09-29,10588707,77.744203
2186,2022-09-29,10626610,270.357196
2187,2022-09-30,1000000603,17.927143
2188,2022-09-30,10588707,73.335098


In [264]:
from numpy import datetime64, arange

In [317]:
def generate_consumption(meters: pd.DataFrame, start_date: str, end_date: str) -> pd.DataFrame:
    """Generates random mock consumption data."""
    
    # convert dates to numpy dates
    start_date=datetime64(start_date)
    end_date=datetime64(end_date)
    
    # dates for every day between start and end
    dates=pd.DataFrame({"Date": arange(start_date, end_date)})
    
    # Now for every date a random consumption for each meter
    result=pd.merge(dates, meters.reset_index(), how="cross")
    
    # Create random consumption in the right order of magnitude:
    # 1/365 of annual rate with 40% variation
    result["kWh"]=result["Annual Quantity (kWh)"].divide(365.25).multiply(.4*random()+.8).round().astype(int)
    
    # we can drop superfluous cols now
    return result.drop(["Exit Zone", "Annual Quantity (kWh)"], axis=1)

In [315]:
generate_consumption(generate_meters(5), "2022-10-01", "2022-11-01")

,Date,Meter ID,kWh
0,2022-10-01,0,2657
1,2022-10-01,1,1952
2,2022-10-01,2,932
3,2022-10-01,3,453
4,2022-10-01,4,2975
...,...,...,...
150,2022-10-31,0,2657
151,2022-10-31,1,1952
152,2022-10-31,2,932
153,2022-10-31,3,453
